# Tutorial: skultrafast.dataset

skultrafast is a package mainly for working with time-resolved spectra. The notebook shows how to
use the new dataset-class to work with such a spectra and to fit it with a exponetial model. First we import
the necessary modules and configure matplotlib to show inline graphics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
from skultrafast.dataset import DataSet
import skultrafast
from skultrafast import data_io
print(skultrafast.__file__)
skultrafast.__version__

## Creating a DataSet
In this tuturial we use the example data which is provided by *skultrafast*. 

In [ ]:
wavelengths, t_ps, data_mOD = data_io.load_example()

Lets look at the constructor of the `DataSet` class:

In [ ]:
?DataSet

As we see, we have all the necessay variables, additionlly we can name the dataset. Since the `freq_unit` defaults to 'nm' we don't need to supply this argument.

In [ ]:
ds = DataSet(wavelengths, t_ps, data_mOD, name="Al(tpfc)(py)_2")

## Overview

To get an general idea of a transient spectra is to plot some data.  All plotting functions are in the `DataSet.plot` object, which is an instance of `DataSetPlotter`. The plotting functions are using the `disp_freq_unit` of the dataset as frequency scale by default. This can be changed by changing the `disp_freq_unit` of the `DataSetPlotter` object. 

In [ ]:
ds.plot.disp_freq_unit = 'nm' # does nothing, since 'nm' is the default
# ds.plot.disp_freq_unit = 'cm' would use wavenumbers

First, we want to check if the dataset is corrected for dispersion. For that we plot a colormap around the time-zero. 

In [ ]:
ds.plot.map(symlog=0, con_step=10., con_filter=(3, 10))
plt.ylim(-2, 2)

Evidently, the dataset is not corrected for dispersion. Since it is easier to work with a dispersion corrected dataset, we try to get an estimate of the dispersion using the data directly. 

*skultrafast* does this by first using a simple heuristic for determining the time-zero for each transient. The resulting dispersion curve is then fitted with a polynomial, using a robust fitting method. More details are given in the documentation.

In [ ]:
# First calculate and plot the estimate.
res = ds.estimate_dispersion(heuristic_args=(1.5,), deg=2)
ds.plot.map(symlog=0, con_step=10., con_filter=(3, 10))
plt.ylim(-2, 2)
plt.plot(ds.wavelengths, res.polynomial(ds.wavenumbers)) #The polynomial is defined in wavenumbers
plt.plot(ds.wavelengths, res.tn)

By default, *skultrafast* uses a very simple heuristic to find the time-zero. It looks for the earliest value above a given limit in each transient, and therefore underestimates the time-zero systematically. Therefore we slightly shift the time-zero. 

This generallay works surprinsingly well. But if the exact time-zero is necessary, I recommend to try other methods or measure the dispersion directly.

**WARNING**: The cell below changes the dataset inplace. Therefore repeated calls to the cell will shift the time-zero again and again.

In [ ]:
new_ds = res.correct_ds #warning, this is no copy
new_ds.t -= 0.2

In [ ]:
new_ds.plot.map(con_step=10., con_filter=(3, 5))

In [ ]:
res.correct_ds.plot.spec([-.2, 0.05, 0.3, 1, 2, 150])

In [ ]:
res.correct_ds.plot.trans([500, 620, 680], symlog=1)

## Exponential fitting


__Note: The section is likly to change, since the current interface is still very rough.__

Fitting a decay-associated spectra is a one-liner in skultrafast. If the dataset is dispersion corrected,
only a starting guess is necessay.

In [ ]:
fit_res = new_ds.fit_exp([-0.0, 0.05, 2, 20, 10000])
fit_res.lmfit_res.params.pretty_print()

### Plotting the DAS, using old skultrafast functions

In [ ]:
plt.plot(new_ds.t-fit_res.lmfit_res.params['p0'].value, fit_res.fitter.model[:, 200])
plt.plot(new_ds.t-fit_res.lmfit_res.params['p0'].value, fit_res.fitter.data[:, 200], 'o', color='k', ms=2)
plt.xlim(-1, 10)

In [ ]:
plt.plot(new_ds.wavelengths, fit_res.fitter.c[:, :3])

In [ ]:
plt.plot(new_ds.t, fit_res.fitter.x_vec)

In [ ]:
f = fit_res.fitter
plt.plot(np.dot(f.c, f.x_vec.T).T[:, 200])

In [ ]:
f.x_